In [1]:
import numpy as np
import tritonclient.http

In [2]:
url = "10.10.66.25:8000"
model_name = "speech-video-handler"
model_version = "1"
batch_size = 1

In [22]:
triton_client = tritonclient.http.InferenceServerClient(url=url, verbose=False)

In [4]:
triton_client.is_model_ready(model_name=model_name, model_version=model_version)

True

In [5]:
model_metadata = triton_client.get_model_metadata(model_name=model_name, model_version=model_version)

In [6]:
model_config = triton_client.get_model_config(model_name=model_name, model_version=model_version)

### Video handler requests

In [7]:
_input = tritonclient.http.InferInput(name="video_url", shape=(batch_size,), datatype="BYTES")
output_1 = tritonclient.http.InferRequestedOutput(name="AUDIO_TEXT", binary_data=False)
# output_2 = tritonclient.http.InferRequestedOutput(name="LANGUAGE", binary_data=False)

In [8]:
video_url = "https://cdn-st.rutubelist.ru/media/d1/e7/642dc2194fcdb69664f832d5f2dd/fhd.mp4"

In [9]:
_input.set_data_from_numpy(np.asarray([video_url] * batch_size, dtype=object))

response = triton_client.infer(
        model_name=model_name,
        model_version=model_version,
        inputs=[_input],
        outputs=[output_1],
    )

print(response.get_response())

{'model_name': 'speech-video-handler', 'model_version': '1', 'outputs': [{'name': 'AUDIO_TEXT', 'datatype': 'BYTES', 'shape': [], 'data': ['я убежден что чем беднее человек тем сложнее его удовлетворить потому что тот человек у которого нет денег он всегда недоволен чем он всегда просит больше который в принципе удовлетворен своими деньгами он говорит классно это здорово спасибо и идет и делает']}]}


In [10]:
np.array2string(response.as_numpy("AUDIO_TEXT"))

"'я убежден что чем беднее человек тем сложнее его удовлетворить потому что тот человек у которого нет денег он всегда недоволен чем он всегда просит больше который в принципе удовлетворен своими деньгами он говорит классно это здорово спасибо и идет и делает'"

In [11]:
response.get_output("AUDIO_TEXT")['data'][0]

'я убежден что чем беднее человек тем сложнее его удовлетворить потому что тот человек у которого нет денег он всегда недоволен чем он всегда просит больше который в принципе удовлетворен своими деньгами он говорит классно это здорово спасибо и идет и делает'

### Xclip handler requests

In [12]:
model_name = "xclip-video-handler"

_input = tritonclient.http.InferInput(name="video_url", shape=(batch_size,), datatype="BYTES")
output_1 = tritonclient.http.InferRequestedOutput(name="EMBEDDING", binary_data=False)

In [27]:
# video_url = "https://cdn-st.rutubelist.ru/media/d1/e7/642dc2194fcdb69664f832d5f2dd/fhd.mp4"
video_url = "https://cdn-st.rutubelist.ru/media/82/04/13afdc714bab8f2f5269fc59de01/fhd.mp4"
_input.set_data_from_numpy(np.asarray([video_url] * batch_size, dtype=object))

response = triton_client.infer(
        model_name=model_name,
        model_version=model_version,
        inputs=[_input],
        outputs=[output_1],
    )

print(response.get_response())

{'model_name': 'face-analysis', 'model_version': '1', 'outputs': [{'name': 'EMBEDDINGS', 'datatype': 'FP32', 'shape': [3, 512], 'data': [-1.0573445558547974, 0.7403695583343506, 1.3779935836791992, 0.06294821202754974, -0.335049569606781, 1.6624501943588257, -0.1688232123851776, -0.12059809267520905, -1.226500153541565, -0.8125243186950684, -0.0319996252655983, -0.08302426338195801, -0.3438990116119385, 0.22194552421569824, -0.47084182500839233, 0.3138795793056488, 1.5370402336120605, 0.5647506713867188, 1.0267717838287354, 0.5549613833427429, 0.2521912157535553, 0.8656278848648071, -1.520696759223938, 0.2926270067691803, -0.4151633381843567, 0.8041015863418579, 1.5597045421600342, 0.7657462358474731, -0.7138097882270813, -0.08366404473781586, -0.7230019569396973, -1.2204843759536743, -0.49833279848098755, -0.897665798664093, -0.9387100338935852, -0.6927413940429688, 0.45558470487594604, 0.00026209131465293467, 0.6273133158683777, 0.0509331151843071, 0.15399518609046936, 0.265781402587

In [19]:
triton_client.close()

### Face embeddings model

In [20]:
model_name = "face-analysis"

_input = tritonclient.http.InferInput(name="video_url", shape=(batch_size,), datatype="BYTES")
output_1 = tritonclient.http.InferRequestedOutput(name="EMBEDDINGS", binary_data=False)

In [23]:
video_url = "https://cdn-st.rutubelist.ru/media/d1/e7/642dc2194fcdb69664f832d5f2dd/fhd.mp4"
_input.set_data_from_numpy(np.asarray([video_url] * batch_size, dtype=object))

response = triton_client.infer(
        model_name=model_name,
        model_version=model_version,
        inputs=[_input],
        outputs=[output_1],
    )

print(response.get_response())

{'model_name': 'face-analysis', 'model_version': '1', 'outputs': [{'name': 'EMBEDDINGS', 'datatype': 'FP32', 'shape': [1, 512], 'data': [-0.39640986919403076, 0.5577621459960938, -0.38480040431022644, -0.5430930852890015, -0.5977988243103027, 0.8291841745376587, -0.8572166562080383, -0.4391906261444092, 1.5082900524139404, 0.571452260017395, -1.0145152807235718, 0.41790148615837097, 1.493511438369751, -0.4966958463191986, 0.20283396542072296, 0.7836390733718872, -0.40304818749427795, 0.006415354087948799, 0.31307706236839294, 0.73674076795578, -0.16823141276836395, 0.41729435324668884, -1.0762966871261597, -0.09193019568920135, -1.0771055221557617, -0.725462019443512, 0.2510703206062317, 1.4907912015914917, 0.13848771154880524, -0.97098308801651, -1.2592426538467407, -0.3069590628147125, -0.704663097858429, -0.6280113458633423, -0.11498209089040756, -0.5642503499984741, -0.2103513777256012, -0.4158037304878235, -0.23870427906513214, -1.6885080337524414, 0.6630844473838806, 0.9056518077

In [28]:
response.as_numpy("EMBEDDINGS").shape

(3, 512)